Imports

In [14]:
from time import sleep
import pickle, json, pyautogui

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager


WebDriver Initialization

In [15]:
options = Options()
options.add_argument("start-maximized")
# options.add_argument(
#     {"user-data-dir": r"C:\Users\shiva\AppData\Local\Google\Chrome\User Data\Default"}
# )

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)
pyautogui.hotkey('win','d')
pyautogui.hotkey('alt','tab')
pyautogui.hotkey('alt','tab')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\shiva\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Login Sequence

In [3]:
# driver.get("https://www.swiggy.com")
# driver.find_element(by=By.LINK_TEXT, value="Login").click()
# driver.find_element(by=By.NAME, value="mobile").send_keys("9318335215")
# driver.find_element(by=By.NAME, value="mobile").send_keys(Keys.ENTER)
# sleep(20)
# driver.find_element(by=By.CLASS_NAME, value="_24Etq").click()


Location Sequence

In [4]:
driver.get("https://swiggy.com")
driver.find_element(by=By.NAME, value="location").send_keys(
    "Dwarka Sector 11, Dwarka, Delhi, India"
)
sleep(2)
driver.find_element(by=By.CLASS_NAME, value="_2W-T9").click()
sleep(2)


Scroll to the Bottom

In [5]:
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo({top: document.body.scrollHeight-7000,left: 0,behavior: 'smooth'});")

    # Wait to load page
    sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


Create Restraunt's Links List

In [6]:
no_of_restraunts = driver.find_element(by=By.CLASS_NAME, value="BZR3j").text[0:-12]
restraunt_elements_list = driver.find_elements(by=By.CSS_SELECTOR, value=".undefined a")
restraunt_links_list = []
for restraunt_element in restraunt_elements_list:
    if (
        restraunt_element.get_attribute("href") != "https://www.swiggy.com/search"
        and restraunt_element.get_attribute("href") != None
    ):
        restraunt_links_list.append(restraunt_element.get_attribute("href"))

print(f"Restraunts Found: {len(restraunt_links_list)}/{no_of_restraunts}")

with open("restraunt_links_list.log","wb") as file1:
    pickle.dump(restraunt_links_list, file1)


Restraunts Found: 63/1331


Verify Restraunt's Links List

In [16]:
with open("restraunt_links_list.log","rb") as file1:
    restraunt_links_list=pickle.load(file1)
print(f"Links Saved: {len(restraunt_links_list)}")
# print(restraunt_links_list)

Links Saved: 63


Collect Data

In [29]:
data = []
for restraunt_number in range(0, 1):
    restraunt_data = {}

    driver.get(restraunt_links_list[restraunt_number])

    restraunt_data["name"] = driver.find_element(by=By.CLASS_NAME, value="_3aqeL").text
    restraunt_data["category"] = (
        driver.find_element(by=By.CLASS_NAME, value="JMACF").text
    ).split(", ")
    restraunt_data["address"] = driver.find_element(
        by=By.CLASS_NAME, value="_2Y6HW"
    ).text

    info_list = driver.find_elements(by=By.CSS_SELECTOR, value="._2fC4N div div span")

    restraunt_data["rating"] = info_list[0].text
    restraunt_data["delivery time"] = info_list[3].text[:-5]
    restraunt_data["average_cost"] = info_list[4].text[2:]

    

    data.append(restraunt_data)

json_data=json.dumps(data, indent=4)
print(json_data)


[
    {
        "name": "Caterspoint",
        "category": [
            "Italian",
            "Indian"
        ],
        "address": "Dwarka, Sector 7, Dwarka",
        "rating": "4.2",
        "delivery time": "--",
        "average_cost": "600"
    }
]


Store Data to JSON File

In [28]:
with open("data.json", "w") as file2:
    json.dump(data, file2)

[
    {
        "name": "Caterspoint",
        "category": [
            "Italian",
            "Indian"
        ],
        "address": "Dwarka, Sector 7, Dwarka",
        "rating": "4.2",
        "delivery time": "--",
        "average_cost": "600"
    }
]


Quit WebDriver

In [10]:
driver.quit()

Test

In [9]:
data = []
restraunt_data = {}
driver.get(
    "https://www.swiggy.com/restaurants/the-belgian-waffle-co-global-foyer-sector-1-gurgaon-412735"
)

restraunt_data["name"] = driver.find_element(by=By.CLASS_NAME, value="_3aqeL").text
restraunt_data["category"] = (
    driver.find_element(by=By.CLASS_NAME, value="JMACF").text
).split(",")
restraunt_data["address"] = driver.find_element(by=By.CLASS_NAME, value="_2Y6HW").text

info_list = driver.find_elements(by=By.CSS_SELECTOR, value="._2fC4N div div span")

restraunt_data["rating"] = info_list[0].text
restraunt_data["delivery time"] = info_list[3].text
restraunt_data["average_cost"] = info_list[4].text[2:]

restraunt_data["menu"] = {
    categories: [
        {
            "category_name": "Icecream",
            "items": [
                {
                    "item_name": "chocolate",
                    "price": "500",
                    "desc": "description of item",
                },
                {"item_name": "vanilla", "price": "500", "desc": "description of item"},
            ],
        },
        {
            "category_name": "Food",
            "items": [
                {"item_name": "Paneer", "price": "500", "desc": "description of item"},
                {"item_name": "Dal", "price": "500", "desc": "description of item"},
            ],
        },
    ]
}

category_list = driver.find_elements(by=By.CSS_SELECTOR, value="._1J_la ._2dS-v .M_o7R")
sub_category_list = driver.find_elements(
    by=By.CSS_SELECTOR, value="._1J_la ._2dS-v ._1Jgt5 ._2WzQq"
)
category_id_list = driver.find_elements(by=By.CSS_SELECTOR, value="._1J_la ._2dS-v")

data.append(restraunt_data)
print(data)


[{'name': 'The Belgian Waffle Co.', 'category': ['Opens next at 10 am', ' todayDesserts', ' Ice Cream'], 'address': 'Global Foyer, Sector 1', 'rating': '4.3', 'delivery time': 'Closed', 'average_cost': '200'}]


In [30]:
dummy_list = driver.find_elements(by=By.CSS_SELECTOR, value="._1J_la ._2dS-v")
dummy_list2 = []
for x in dummy_list:
    y = x.get_attribute("id")
    # dummy_list3 = driver.find_elements(
    #     by=By.CSS_SELECTOR,
    #     value=f"._1J_la #{y} ._2wg_t p",
    # )
    # dummy_list2+=dummy_list3
    print(y)


h-1950595611
h462583133
h-148333512
h-1210827099
h-1438569101
h-1108832219
h921735350
h-2085977779
h-478071696
h-22090143
h-580657181
h-1730620042
h1550842745


In [63]:
dummy_list = driver.find_elements(
    by=By.CSS_SELECTOR,
    value="#h-1308881351 ._1Jgt5 ._2wg_t .styles_detailsContainer__22vh8",
)
for x in dummy_list:
    print(x.text)


Honey Butter Waffle
107.6230% OFF | USE TRYNEW
Classic crispy waffle + Warm melted butter with a golden honey drizzle. An American classic!
Bestseller
Maple Butter Waffle
107.6230% OFF | USE TRYNEW
Classic crispy waffle + Warm melted butter with maple syrup drizzle. An American classic!
Strawberry Cream Cheese Waffle
165.2430% OFF | USE TRYNEW
Classic crispy waffle + Strawberry compote layered with Philadelphia style cream cheese. Deliciously tart!
Blueberry Cream Cheese Waffle
174.7630% OFF | USE TRYNEW
Classic crispy waffle + Blueberry compote layered with Philadelphia style cream cheese. Deliciously tart!
Bestseller
Belgian Chocolate Milk Waffle
155.7130% OFF | USE TRYNEW
Classic crispy waffle + Melted Belgian milk chocolate. Simplicity at its best!
Belgian Chocolate Dark Waffle
155.7130% OFF | USE TRYNEW
Classic crispy waffle + Melted Belgian dark chocolate. Darker simplicity!
67% Less sugar Chocolate Overload Milk Waffle
175.7130% OFF | USE TRYNEW
Double the Chocolate, less than h

Final Code!!!!!

In [73]:
category_element_list = driver.find_elements(
    by=By.CSS_SELECTOR, value="._1J_la ._2dS-v"
)
reccomended_category = category_element_list[0].get_attribute("id")
food_item_element_list2 = driver.find_elements(
    by=By.CSS_SELECTOR,
    value=f"#{reccomended_category} .styles_detailsContainer__22vh8",
)
for food_item_name in food_item_element_list2:
    print(food_item_name.text)
    print()

for category_element in category_element_list:
    category_id = category_element.get_attribute("id")
    # print(driver.find_element(by=By.CSS_SELECTOR, value=f"#{category_id} .M_o7R").text)

    # for x in driver.find_elements(by=By.CSS_SELECTOR, value=f"#{category_id} ._1Jgt5 ._2WzQq"):
    #     print("\t"+x.text)

    sub_category_element_list = driver.find_elements(
        by=By.CSS_SELECTOR, value=f"._1J_la #{category_id} ._1Jgt5"
    )
    for sub_category_element in sub_category_element_list:
        sub_category_id = sub_category_element.get_attribute("id")
        print(sub_category_id)
        food_item_element_list = driver.find_elements(
            by=By.CSS_SELECTOR,
            value=f"#{category_id} #{sub_category_id} .styles_detailsContainer__22vh8",
        )
        for food_item_name in food_item_element_list:
            print(food_item_name.text)
            print()


Double Delight Sundae combo
34930% OFF | USE TRYNEW
Save Rs. 31 flat and enjoy the delicious Waffle Sundaes. Combo Includes Rocky Road Sundae & Triple Cookie Sundae

Happiness in mini version + 1 FREE Delight
511.1930% OFF | USE TRYNEW
Buy Chocolate Mini Waffle Combo pack of 6 and select 1 FREE waffle or pancake from our top picks

Keep Cake & Celebrate + 1 FREE Delight
580.5230% OFF | USE TRYNEW
Choose any one double layer waffle cake- Death by chococolate Waffle Cake (Double Layer)/ Red velvet waffle Cake (Double Layer)/ Kiki and Oreo Cream waffle cake (Double Layer) and select 1 FREE waffle or pancake from our top picks

Buy 3 Belgian Chocolate Milk Waffles and Get 1 Free Delight
467.1430% OFF | USE TRYNEW
Buy Belgian chocolate [Milk] waffle and select 1 FREE waffle or pancake from our top picks.

Buy 3 Belgian Chocolate Dark Waffles and Get 1 Free Delight
467.1430% OFF | USE TRYNEW
Buy Belgian chocolate [Dark] waffle and select 1 FREE waffle or pancake from our top picks.

Triple F

In [61]:
category_name_element_list = driver.find_elements(
    by=By.CSS_SELECTOR, value="._1J_la ._2dS-v .M_o7R"
)
food_name_element_list = driver.find_elements(
    by=By.CSS_SELECTOR,
    value="#h1223869201 .styles_detailsContainer__22vh8 .styles_itemNameText__3ZmZZ",
)
food_price_element_list = driver.find_elements(
    by=By.CSS_SELECTOR, value="#h1223869201 .styles_detailsContainer__22vh8 .rupee"
)
food_description_element_list = driver.find_elements(
    by=By.CSS_SELECTOR,
    value="#h1223869201 .styles_detailsContainer__22vh8 .styles_itemDesc__3vhM0",
)

for food_name_element in food_name_element_list:
    food_name = food_name_element.text
    print(food_name)
    print()


Double Delight Sundae combo

Happiness in mini version + 1 FREE Delight

Keep Cake & Celebrate + 1 FREE Delight

Buy 3 Belgian Chocolate Milk Waffles and Get 1 Free Delight

Buy 3 Belgian Chocolate Dark Waffles and Get 1 Free Delight

Triple Flavour Festivities + 1 FREE Delight

Mini Waffles party pack

Best of the best combo

Loaded Waffle combo

Triple Batter Bliss (Combo of 3)

Triple Chocolate Waffle + Belgian Chocolate Shake (Combo of 2)

Naked Nutella Waffle + Naked Nutella Pancake (Combo of 2)



In [51]:
restraunt_data["menu"] = {
    categories: [
        {
            "category_name": "Icecream",
            "items": [
                {
                    "item_name": "chocolate",
                    "price": "500",
                    "desc": "description of item",
                },
                {"item_name": "vanilla", "price": "500", "desc": "description of item"},
            ],
        },
        {
            "category_name": "Food",
            "items": [
                {"item_name": "Paneer", "price": "500", "desc": "description of item"},
                {"item_name": "Dal", "price": "500", "desc": "description of item"},
            ],
        },
    ]
}


In [ ]:
category_name_element_list = driver.find_elements(
    by=By.CSS_SELECTOR, value="._1J_la ._2dS-v .M_o7R"
)
food_name_element_list = driver.find_elements(
    by=By.CSS_SELECTOR,
    value="#h1223869201 .styles_detailsContainer__22vh8 .styles_itemNameText__3ZmZZ",
)
food_price_element_list = driver.find_elements(
    by=By.CSS_SELECTOR, value="#h1223869201 .styles_detailsContainer__22vh8 .rupee"
)
food_description_element_list = driver.find_elements(
    by=By.CSS_SELECTOR,
    value="#h1223869201 .styles_detailsContainer__22vh8 .styles_itemDesc__3vhM0",
)


In [66]:
category_element_list = driver.find_elements(by=By.CSS_SELECTOR, value="._1J_la ._2dS-v")
for category_element in category_element_list:
    (category_element.get_attribute("id")

h-1950595611
h1223869201
h-1308881351
h1426971542
h1921240031
h-410127158
h2136257937
h-1985464008
h98949598
